In [1]:
# import necessary libraries
from selenium import webdriver
import undetected_chromedriver as uc
import chromedriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re

C:\Users\Hii\AppData\Local\Temp\ipykernel_12568\2823494389.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
vehicleResults = pd.DataFrame(columns=['branch', 'model', 'price','yearOfConstruction','mileStone','transmission','cilinders','prevOwners'])
def extract_price_value(price):
    pattern = r'([\d,]+(?:\.\d+)?)'
    match = re.search(pattern, price)

    if match:
        numeric_value = match.group(1).replace(',', '')
        numeric_value = float(numeric_value)
        return numeric_value
    else:
        return '0'

def extract_numeric_value(string):
    # Remove any non-digit characters except the comma
    numeric_string = ''.join(char for char in string if char.isdigit() or char == ',')

    # Remove the comma from the numeric string
    numeric_string = numeric_string.replace(',', '')

    # Convert the numeric string to an integer
    numeric_value = int(numeric_string)

    return numeric_value

def extract_year(date):
    # Split the date into month and year parts
    month, year = date.split('/')

    # Extract the year part
    year = int(year)

    return year
    
def getValueFromKeyFeatureDiv(selector):
    divEle = driver.find_element(By.CSS_SELECTOR, selector)
    return divEle.find_element(By.CSS_SELECTOR, "div.key-feature__value").text

def scrapeVehicleData():

    global vehicleResults

    #Wait page load
    time.sleep(10)
    #Init data
    vehicleData = {}

    #Get branch and model
    branchModelTitle = driver.find_element(By.ID, "ad-title").text
    branchModelSplitArray = branchModelTitle.split()
    branch = branchModelSplitArray[0]
    model = ' '.join(branchModelSplitArray[1:])
    
    #Get price
    price = driver.find_element(By.CSS_SELECTOR, "span[data-testid='prime-price']").text

    #Get year of construction
    yearOfConstruction = getValueFromKeyFeatureDiv("div.key-feature--firstRegistration")

    #Get mileStone
    mileStone = getValueFromKeyFeatureDiv("div.key-feature--mileage")

    #Get transmission
    transmission = getValueFromKeyFeatureDiv("div.key-feature--transmission")

    #Get cilinders
    cilinders = driver.find_element(By.ID, "cubicCapacity-v").text

    #Get prevOwners
    prevOwners = '0'
    try:
        prevOwners = getValueFromKeyFeatureDiv("div.key-feature--numberOfPreviousOwners").text
    except:
        print("Vehicle doesn't have preowners")    

    vehicleData["branch"] = branch
    vehicleData["model"] = model
    vehicleData["price"] = extract_price_value(price)
    vehicleData["yearOfConstruction"] = extract_year(yearOfConstruction)
    vehicleData["mileStone"] = extract_numeric_value(mileStone)
    vehicleData["transmission"] = transmission
    vehicleData["cilinders"] = extract_numeric_value(cilinders)
    vehicleData["prevOwners"] = prevOwners

    vehicleResults = pd.concat([vehicleResults, pd.DataFrame([vehicleData])], ignore_index=True)

    print("VehicleData: ", vehicleData)
    return vehicleData

def scrapeFromVehicleLinks():
    container = driver.find_element(By.XPATH, "//article[@data-testid='result-list-container']")
    # Find all <a> tags with href starting with "/fahrzeuge/details.html" inside the container
    links = container.find_elements(By.CSS_SELECTOR, "a[href^='/fahrzeuge/details.html']")
    size = len(links)
    step = 250
    for x in range(size):
        driver.execute_script(f"window.scrollBy(0,{step})")
        print("Link: ", links[x].get_attribute("href"))
        links[x].click()
        # driver.get(link.get_attribute("href"))

        #Focus to the detail vehicle tab
        driver.switch_to.window(driver.window_handles[1])

        #Get data
        # print("Scraped data:", scrapeVehicleData())
        scrapeVehicleData()

        driver.close()

        driver.switch_to.window(driver.window_handles[0])

        #Wait page load
        time.sleep(10)

        links = container.find_elements(By.CSS_SELECTOR, "a[href^='/fahrzeuge/details.html']")
        step += 250

def getPageNumberSearchLink(pageNumber):
    return f'https://suchen.mobile.de/fahrzeuge/search.html?cn=DE&door=FOUR_OR_FIVE&ecol=BLACK&ecol=GREY&ecol=SILVER&ecol=WHITE&ft=PETROL&gn=45899%2C+Gelsenkirchen%2C+Nordrhein-Westfalen&isSearchRequest=true&ll=51.5308428%2C7.0328265&ml=%3A200000&p=%3A7500&pageNumber={pageNumber}&rd=100&s=Car&sb=rel&vc=Car'        

# webdriver_manager = ChromeDriverManager().install()
options = webdriver.ChromeOptions()
version_main = int(chromedriver_autoinstaller.get_chrome_version().split(".")[0])
#driver_executable_path=webdriver_manager
driver = uc.Chrome(headless=False,use_subprocess=False,version_main=version_main,options=options,driver_executable_path="C:/Users/Hii/Downloads/chromedriver-win64/chromedriver.exe")
driver.get("https://suchen.mobile.de/?lang=en")

driver.delete_all_cookies()

acceptBtn = driver.find_element(By.XPATH, "//button[text()='Accept']")
acceptBtn.click()

pageNum = 1

time.sleep(5)

driver.execute_script(f"window.scrollBy(0,{600})")
time.sleep(5)
driver.execute_script(f"window.scrollBy(0,{200})")
time.sleep(5)
driver.execute_script(f"window.scrollBy(0,{1000})")

driver.get(getPageNumberSearchLink(pageNum))

while True:
    try:
        scrapeFromVehicleLinks()
        pageNum += 1
        driver.get(getPageNumberSearchLink(pageNum))
    except Exception as e:
        print("Exception when trying to get next page", e)
        break

vehicleResults.to_csv('output.csv', index=False)



    

    
    





Exception when trying to get next page Message: no such element: Unable to locate element: {"method":"xpath","selector":"//article[@data-testid='result-list-container']"}
  (Session info: chrome=121.0.6167.86)
Stacktrace:
	GetHandleVerifier [0x00651673+52979]
	(No symbol) [0x005D7961]
	(No symbol) [0x004BDD3D]
	(No symbol) [0x004F5FBB]
	(No symbol) [0x004F60FB]
	(No symbol) [0x0052CF92]
	(No symbol) [0x00514534]
	(No symbol) [0x0052B3FE]
	(No symbol) [0x00514286]
	(No symbol) [0x004EC063]
	(No symbol) [0x004ECECD]
	GetHandleVerifier [0x00968D83+3294723]
	GetHandleVerifier [0x009A6CC2+3548482]
	GetHandleVerifier [0x009A1C9C+3527964]
	GetHandleVerifier [0x006E870E+671630]
	(No symbol) [0x005E1EB4]
	(No symbol) [0x005DD808]
	(No symbol) [0x005DD92D]
	(No symbol) [0x005CF7E0]
	BaseThreadInitThunk [0x76A0FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77997C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77997C3E+238]



In [29]:
# Create ChromeOptions object
options = webdriver.ChromeOptions()
driver = webdriver.Chrome()
driver.get("http://Mobile.de?lang=en")
acceptBtn = driver.find_element(By.XPATH, "//button[text()='Accept']").text

acceptBtn.click()

resultsBtn = driver.find_element(By.XPATH, "//button[@data-testid='qs-submit-button']")
# time.sleep(10)
resultsBtn.click()

resultsBtn.get_attribute("href")

driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.COMMAND + 't') 

